In [1]:
import pandas as pd

In [4]:
weather_df = pd.read_csv('../data/internal/weather.csv')

In [6]:
weather_df = weather_df.drop("Rain", axis='columns')

In [8]:
print(weather_df.dtypes)

Country        object
Name           object
Weather        object
Temp          float64
Humidity        int64
Visibility      int64
Wind speed    float64
Clouds          int64
dtype: object
